# <span style="color:gold">**Análisis descriptivo usando Pandas**</span>
***

### **Editado por: Kevin Alexander Gómez**
#### Contacto: kevinalexandr19@gmail.com | [Linkedin](https://www.linkedin.com/in/kevin-alexander-g%C3%B3mez-2b0263111/) | [Github](https://github.com/kevinalexandr19)
***

### **Descripción**

En este tutorial, revisaremos un flujo de trabajo Python orientado al <span style="color:gold">análisis descriptivo de datos</span>.

Este Notebook es parte del proyecto [**Python para Geólogos**](https://github.com/kevinalexandr19/manual-python-geologia), y ha sido creado con la finalidad de facilitar el aprendizaje en Python para estudiantes y profesionales en el campo de la Geología.

## **Índice**
***

1. [¿Qué es el análisis descriptivo?](#parte1)
2. [Ventajas de analizar datos en Python](#parte2)
2. [Ejemplo de análisis descriptivo en Python](#parte3)

***

<a id="parte1"></a>

## **1. ¿Qué es el análisis descriptivo?**
***

De acuerdo con el [Glosario IT de Gartner](https://www.gartner.com/en/information-technology/glossary/descriptive-analytics#:~:text=Descriptive%20Analytics%20is%20the%20examination,%2C%20tables%2C%20or%20generated%20narratives.), el análisis descriptivo es la primera fase del **análisis de datos**.\
Consiste en **examinar** los datos con el fin de responder a la pregunta **¿qué ha sucedido?** o **¿qué está sucediendo?**.

En esta fase, los datos generados en tiempo real e históricos se analizan a través de métodos estadísticos simples y técnicas de visualización.

<img src="resources/analytic_value_escalator.jpg" alt="Las 4 fases en el análisis de datos" width="700"/>

> Es común reportar los resultados de esta fase a través de reportes, resúmenes o dashboards.

También es importante entender los **tipos de datos** con los que tratamos comúnmente en un análisis estadístico:
- **Datos categóricos:** información no numérica como por ejemplo:
    - *litología*,
    - *dominio geológico*,
    - *textura de roca*, etc.


- **Datos numéricos:** medidas o conteos como por ejemplo:
    - *número de muestras*,
    - *concentración elemental*,
    - *longitud*, etc.

<a id="parte2"></a>

## **2. Ventajas de analizar datos en Python**
***
- Aprender a programar te permite desarrollar tus habilidades analíticas y de resolución de problemas.
- Puedes crear e implementar diferentes herramientas de manera independiente y usarlas en cada etapa del análisis de datos.
- <span style="color:gold">Python</span> es uno de los lenguajes más usados en <span style="color:lightgreen">Ciencia de Datos</span> debido a su sintáxis sencilla y clara escritura.
- Todos los trabajos realizados en <span style="color:gold">Python</span> son flexibles y escalables.

<a id="parte3"></a>

## **3. Ejemplo de análisis descriptivo en Python**
***

Usaremos un archivo Excel perteneciente a la base de datos del **IGP (Instituto Geofísico del Perú)**.\
Específicamente, la información que contiene describe la *ocurrencia de terremotos en Perú desde 1960*.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="ticks", context="talk") # Mejor calidad visual
plt.style.use("dark_background") # Gráficos con fondo oscuro

### **3.1. Procesando la información**

Empezaremos usando la función `read_excel` para leer el archivo ubicado en la carpeta `files`.\
Asignaremos la información a un DataFrame de nombre `data`.

In [ ]:
data = pd.read_excel("files/sismos.xlsx")

El tamaño de filas y columnas en la tabla se puede observar usando el atributo `shape`:

In [ ]:
data.shape

La tabla contiene 6 columnas y casi 19,000 filas.

Podemos observar las primeras 5 filas de `data` usando el método `head`:

In [ ]:
data.head()

Y las 5 últimas filas usando el método `tail`:

In [ ]:
data.tail()

Observaremos el tipo de datos de cada columna usando el atributo `dtypes`:

In [ ]:
data.dtypes

Notaremos que la tabla contiene los siguientes datos:

- `fecha UTC` : fecha de ocurrencia del terremoto, en **string**.
- `hora UTC` : hora de ocurrencia del terremoto, en **string**.
- `latitud (º)` : latitud del epicentro, en **float**.
- `longitud (º)` : longitud del epicentro, en **float**.
- `profundidad (km)` : profundidad del epicentro, en **integer**.
- `magnitud (M)` : magnitud del terremoto, en **float**.

No usaremos la columna de hora, así que la eliminaremos usando el método `drop`.\
Activaremos el parámetro `inplace=True` para que los cambios se guarden en la tabla.

In [ ]:
data.drop(columns=["hora UTC"], inplace=True)

In [ ]:
data.head()

Ahora, renombraremos las columnas para facilitar su uso dentro del código.\
Primero, crearemos un diccionario que contenga los nombres de las columnas y los nombres que usaremos para reemplazarlos:

In [ ]:
cols = dict(zip(list(data.columns), ["Fecha", "Latitud", "Longitud", "Profundidad", "Magnitud"]))

Ahora, usaremos el método `rename`:

In [ ]:
data.rename(columns=cols, inplace=True)

In [ ]:
data.head()

Ahora, transformaremos el dato de las fechas de **string** a **datetime**:
> **Nota: datetime** es un objeto en Python que representa una fecha.

Seleccionamos la columna con las fechas:

In [ ]:
fecha = data["Fecha"]

In [ ]:
fecha.head(5)

Y usaremos la función `to_datetime` para transformarlo en **datetime**:

In [ ]:
data["Fecha"] = pd.to_datetime(fecha, format="%d/%m/%Y")

Verificamos:

In [ ]:
data.head()

In [ ]:
data.dtypes

### **3.2. Ocurrencia de terremotos cercanos a Lima, en un radio de 150 kilómetros.**

Usaremos la librería `geopy` para seleccionar aquellos epicentros que se encuentren dentro del radio de 150 km:

In [ ]:
from geopy.distance import geodesic

Empezaremos asignando un punto de referencia para la ciudad de Lima:

In [ ]:
punto_lima = [-12.045976, -77.030555]

Seleccionaremos la longitud y latitud de cada epicentro:

In [ ]:
latlon = data[["Latitud", "Longitud"]]

In [ ]:
latlon.head()

Y crearemos una copia de la tabla original usando el método `copy` y asignando la tabla nueva a la variable `lima`:

In [ ]:
data[data["Profundidad"] > 100]

In [ ]:
lima = data[latlon.apply(lambda punto: True if geodesic(punto_lima, punto).km <= 150 else False, axis=1)].copy()

Observamos la nueva tabla:

In [ ]:
lima.head()

También podemos ver un resumen estadístico de la tabla usando el método `describe`:

In [ ]:
lima.describe(datetime_is_numeric=True)

Al haber seleccionado un subconjunto, los índices no empiezan en orden. Usaremos el método `reset_index` para ordenarlos:

In [ ]:
lima.reset_index(drop=True, inplace=True)

In [ ]:
lima.head()

Y mostramos el registro de sismos cercanos a Lima:

In [ ]:
print(f"Se registraron {len(lima)} sismos en un radio de 150 km en torno a Lima.")

> Colocaremos 3 puntos adicionales en el gráfico: Ancón, Cañete y Huacho.\
> Crearemos un diccionario que contenga el nombre de estas ciudades y sus respectivas coordenadas.

In [ ]:
punto_ancon = [-11.773241, -77.176009]
punto_cañete = [-13.077772, -76.387440]
punto_huacho = [-11.108526, -77.610365]

puntos = [punto_lima, punto_ancon, punto_cañete, punto_huacho]
nombres = ["Lima", "Ancón", "Cañete", "Huacho"]

In [ ]:
ciudades = dict(zip(nombres, puntos))

In [ ]:
print(ciudades)

Verificaremos que los sismos hayan sido seleccionados correctamente (i.e. dentro del radio de 150 km).\
Para esto, crearemos una figura usando la función `scatter`:

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10), subplot_kw={"aspect": 1})
ax.scatter(lima["Longitud"], lima["Latitud"], c="white", s=5)

for ciudad, coordenada in ciudades.items():
    ax.scatter(coordenada[1], coordenada[0], c="r", s=25)
    ax.annotate(ciudad, coordenada[::-1], fontsize=18, c="gold")

ax.set_title("Epicentros alrededor de Lima\n (Radio de 150 km)", fontsize=20)
ax.set_xlabel("Longitud (°)", fontsize=18)
ax.set_ylabel("Latitud (°)", fontsize=18)
ax.grid(linewidth=0.5, alpha=0.5)

plt.show()

Usando la información presente y un gráfico para cada caso, explicar:

- ¿Cuántos terremotos ocurrieron por año?

- ¿Con qué frecuencia ocurrieron terremotos en cada mes? ¿Qué mes tiene la mayor frecuencia de terremotos?

- ¿Cuál es la distribución de profundidad de los sismos sobre el área de Lima?



### **3.3. ¿Cuántos terremotos ocurrieron por año?**

Empezaremos revisando los datos generales:

In [ ]:
lima.head()

Para observar la frecuencia de terremotos que ocurrieron por año, usaremos un **diagrama de barras**.\
Para determinar la **frecuencia de terremotos por año**, tendremos que agrupar los datos de acuerdo al año en que ocurrieron.\
Empezaremos extrayendo el año de ocurrencia de cada terremoto. Usaremos el atributo `year` de la columna `Fecha`:

In [ ]:
lima["Fecha"].apply(lambda fecha: fecha.year)

Y la asignaremos a una nueva columna llamada `Año`:

In [ ]:
lima["Año"] = lima["Fecha"].apply(lambda fecha: fecha.year)

Revisando nuevamente los datos generales:

In [ ]:
lima.head()

Ahora, agruparemos los terremotos de acuerdo al año de ocurrencia.\
Para esto, usaremos el método `groupby` para agrupar los datos y `count` para obtener las frecuencias por año.

In [ ]:
lima.groupby(["Año"])["Año"].count().head()

Asignaremos esta información a una variable llamada `sismos`:

In [ ]:
sismos = lima.groupby(["Año"])["Año"].count()

Crearemos una lista con los años de ocurrencia:

In [ ]:
años = list(sismos.index)

Y asignaremos la frecuencia por año en una variable llamada `frecuencia`:

In [ ]:
frecuencia = list(sismos.values)

Para terminar, usaremos la función `bar` y los datos en `frecuencia` y `años` para crear el diagrama de barras:

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))
ax.bar(años, frecuencia, color="lightgreen", alpha=0.75, edgecolor="black")

ax.set_title("Frecuencia de terremotos por año\n Fuente: IGP", fontsize=20)
ax.set_xlabel("Año", fontsize=20)
ax.set_ylabel("Frecuencia", fontsize=18)

ax.set_xticks([1965, 1975, 1985, 1995, 2005, 2015])

plt.show()

### **3.4. ¿Con qué frecuencia ocurrieron terremotos en cada mes? ¿Qué mes tiene la mayor frecuencia de terremotos?**

Para observar la frecuencia de terremotos por mes, usaremos nuevamente un **diagrama de barras**.\
Para determinar la **frecuencia de terremotos por mes**, tendremos que agrupar los datos de acuerdo al mes en que ocurrieron.\
Empezaremos extrayendo el mes de ocurrencia de cada terremoto. Usaremos el atributo `month` de la columna `Fecha`, y lo asignaremos a una nueva columna llamada `Mes`:

In [ ]:
lima["Mes"] = lima["Fecha"].apply(lambda fecha: fecha.month)

Revisamos los datos generales:

In [ ]:
lima.head()

Ahora, agruparemos los terremotos de acuerdo al mes de ocurrencia.\
Para esto, usaremos el método `groupby` para agrupar los datos y `count` para obtener las frecuencias por mes.\
Asignaremos esta información a una variable llamada `sismos`:

In [ ]:
sismos = lima.groupby(["Mes"])["Mes"].count()

Crearemos una lista con los meses de ocurrencia:

In [ ]:
mes = list(sismos.index)

Y otra lista con las frecuencias:

In [ ]:
frecuencia = list(sismos.values)

Ahora, crearemos el diagrama de barras usando la función `bar` y los datos `frecuencia` y `mes`:

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
barra = ax.bar(mes, frecuencia, color="lightgreen", alpha=0.75, edgecolor="black")

ax.set_title("Frecuencia de terremotos por mes\n Fuente: IGP", fontsize=22)

ax.tick_params(left=False, labelleft=False, bottom=False)

nombres_mes = ["Ene", "Feb", "Mar", "Abr", "May", "Jun", "Jul", "Ago", "Sep", "Oct", "Nov", "Dic"]
plt.xticks(ticks=mes, labels=nombres_mes, fontsize=15)

for spine in plt.gca().spines.values():
    spine.set_visible(False)

for bar in barra:
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, str(int(bar.get_height())), ha="center", fontsize=15)
        
plt.show()

### **3.5. ¿Cuál es la distribución de profundidad de los sismos sobre el área de Lima?**

Empezaremos observando la distribución de profundidad a través de un **histograma**.\
Usaremos la columna de `Profundidad` y la función `hist`:

In [ ]:
fig, ax = plt.subplots(figsize=(14, 7))
ax.hist(lima["Profundidad"], bins=30, color="lightgreen", alpha=0.75, edgecolor="black")
ax.set_xticks([10*i for i in range(0, 17)])
plt.show()

Podemos observar que existen dos modas de profundidad de cada sismo: el primero a 50 km y el segundo a 100 km.\
Vamos a separar las profundidades de acuerdo a estos límites.\
Usaremos una función llamada `color_profundidad` para establecer un color de acuerdo a la profundidad de ocurrencia del sismo:

In [ ]:
def color_profundidad(profundidad):
    if profundidad <= 50:
        return "red"
    elif 50 < profundidad <= 100:
        return "gold"
    else:
        return "lightgreen"

Aplicando la función a la columna `Profundidad` y asignando el resultado a una nueva columna llamada `Color`:

In [ ]:
lima["Color"] = lima["Profundidad"].apply(color_profundidad)

Observamos los datos generales:

In [ ]:
lima.head()

Y ahora, graficaremos la distribución de profundidad usando un **diagrama de dispersión** y agregaremos los datos de color generados anteriormente:

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10), subplot_kw={"aspect": 1})
ax.scatter(x=lima["Longitud"], y=lima["Latitud"], color=lima["Color"], s=8)

for ciudad, coordenada in ciudades.items():
    ax.scatter(coordenada[1], coordenada[0], c="b")
    ax.annotate(ciudad, coordenada[::-1], fontsize=15)

ax.set_title("Distribución de profundidad de sismos\n alrededor de Lima (Radio de 150 km)", fontsize=20)
ax.set_xlabel("Longitud (°)", fontsize=15)
ax.set_ylabel("Latitud (°)", fontsize=15)

ax.grid(linewidth=0.5, alpha=0.5)

for color, label in [("red", "Menor a 50 km"), ("gold", "Entre 50 y 100 km"), ("lightgreen", "Mayor a 100 km")]:
    ax.scatter([], [], color=color, label=label)

ax.legend(loc="upper right")

plt.show()

### **3.6. Visualización 3D con `Plotly`**

Usaremos una función llamada `cat_profundidad` para establecer un valor categórico de acuerdo a la profundidad de ocurrencia del sismo:

In [ ]:
def cat_profundidad(profundidad):
    if profundidad <= 50:
        return "Menor a 50 Km"
    elif 50 < profundidad <= 100:
        return "Entre 50 y 100 Km"
    else:
        return "Mayor a 100 Km"

Aplicando la función a la columna `Profundidad` y asignando el resultado a una nueva columna llamada `Categoría`:

In [ ]:
lima["Categoría"] = lima["Profundidad"].apply(cat_profundidad)

In [ ]:
lima.sample(5)

Para terminar, visualizaremos la información en 3D usando `plotly`, una librería de visualización interactiva:

In [ ]:
import plotly.express as px

fig = px.scatter_3d(data_frame=lima, x="Longitud", y="Latitud", z=lima["Profundidad"]*-1, color=lima["Categoría"],
                    color_discrete_map={"Menor a 50 Km": "red", "Entre 50 y 100 Km": "orange", "Mayor a 100 Km": "blue"})

fig.update_layout(
    autosize=False,
    legend = dict(bgcolor="white", title="Profundidad", itemsizing="constant"),
    width=750,
    height=500,
    margin=dict(
        l=50,
        r=50,
        b=50,
        t=50,
        pad=4
    ),
    paper_bgcolor="LightSteelBlue",
    scene = dict(
                    xaxis_title="Longitud",
                    yaxis_title="Latitud",
                    zaxis_title="Profundidad"),
)

fig.update_traces(marker={'size': 1.5})

fig.show()

***